Remove irrelevant columns 

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("../Data/new.csv", encoding="latin1")

C:\Users\jains\AppData\Local\Temp\ipykernel_26416\3915239096.py:1: DtypeWarning: Columns (1,11,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../Data/new.csv", encoding="latin1")


In [4]:
df.columns.tolist()

['url',
 'id',
 'Lng',
 'Lat',
 'Cid',
 'tradeTime',
 'DOM',
 'followers',
 'totalPrice',
 'price',
 'square',
 'livingRoom',
 'drawingRoom',
 'kitchen',
 'bathRoom',
 'floor',
 'buildingType',
 'constructionTime',
 'renovationCondition',
 'buildingStructure',
 'ladderRatio',
 'elevator',
 'fiveYearsProperty',
 'subway',
 'district',
 'communityAverage']

Remove irrelevant columns

In [ ]:
df = df.drop(columns=['url', 'id', 'Cid'])

In [ ]:
df

Check for the categorical coolumns

In [ ]:
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

print("Categorical Columns:")
print(categorical_cols)


Convert date to numerical features

In [ ]:
# Convert tradeTime to datetime
df['tradeTime'] = pd.to_datetime(df['tradeTime'], errors='coerce')

# Extract year and month
df['tradeYear'] = df['tradeTime'].dt.year
df['tradeMonth'] = df['tradeTime'].dt.month

# Drop the original date column
df = df.drop(columns=['tradeTime'])


Handle missing values

In [ ]:
# Numeric imputation with median
for col in df.select_dtypes(include=['float64', 'int64']):
    df[col] = df[col].fillna(df[col].median())

# Categorical imputation with mode
for col in df.select_dtypes(include=['object']):
    df[col] = df[col].fillna(df[col].mode()[0])


Convert all the categorical columns to strings to avoid the mixed data which can't be taken by LabelEncoder

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Identify categorical columns
categorical_cols = df.select_dtypes(include=['object', 'category']).columns

# Ensure all categorical values are strings
df[categorical_cols] = df[categorical_cols].astype(str)

# Apply Label Encoding
le = LabelEncoder()
for col in categorical_cols:
    df[col] = le.fit_transform(df[col])


Encode categorical variables - change into nuerical variable

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Identify categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns

# Apply Label Encoding (example — papers sometimes prefer One-Hot)
le = LabelEncoder()
for col in categorical_cols:
    df[col] = le.fit_transform(df[col])


Outlier handling

In [ ]:
# Outlier capping using IQR method
for col in ['totalPrice', 'price', 'area']:
    if col in df.columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        df[col] = df[col].clip(lower, upper)


Scaling the numeric values - Applied standardization here

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

df[numeric_cols] = scaler.fit_transform(df[numeric_cols])


Saving preprocessed data in .csv format

In [ ]:
# Save the preprocessed data
df.to_csv("../Data/preprocessed.csv", index=False, encoding="utf-8")
print("Preprocessed data saved as preprocessed.csv")
